# UFC Database
- Source of database: https://www.kaggle.com/datasets/rajeevw/ufcdata
- Extracting and manipulating data according to title bouts 

In [130]:
import csv, datetime
import sqlite3
import sqlalchemy

import pandas as pd
import numpy as np

from pandas import Series, DataFrame
from contextlib import closing
from io import StringIO

## Custom functions

## Creating the tables

In [131]:
with open("data.csv", "r") as ufcData:
    ufcDF = pd.read_csv(ufcData)
    ufcDF.rename(
        columns={"date": "Dates"}, inplace=True
    )  # renaming dates column; date keyword is used by sqlite
    ufcDF.index.rename(
        "fighterID", inplace=True
    )  # renaming index column; index keyword is also used by sqlite
    date_time = pd.to_datetime(ufcDF["Dates"], format="%Y-%m-%d")


    '''creating a dataframe with only rows that contain null values'''
    print(ufcDF.isna().sum())


    """creating a dataframe with only title fights"""
    isTitleBout = (
        ufcDF.where(ufcDF["title_bout"] == True)
    ).dropna()  # filters title_bout_checkerDF for title fights
    title_bout_onlyDF = isTitleBout.drop(
        columns=["title_bout"]
    )  # removes the title_bout column
    # print(title_bout_onlyDF)

    """Table 1
    - Name: Title Bouts 
    - Columns to include from title_bout_onlyDF: R_fighter, B_fighter, date, location, 
                                                 Referee, weight_class, Winner
    """
    TitleBouts = pd.DataFrame(
        title_bout_onlyDF,
        columns=[
            "R_fighter",
            "B_fighter",
            "Dates",
            "location",
            "weight_class",
            "Winner",
        ],
    )

    """Table 2a and Table 3a
    - Name: Title Contender Stats (Fight record)
    - Columns to include from title_bout_onlyDF: 
      -> R/B_fighter + stats (Win/Draw/Lose)
    """
    R_TitleFightRec = pd.DataFrame(
        title_bout_onlyDF, columns=["R_fighter", "R_wins", "R_losses", "R_draw"]
    )
    B_TitleFightRec = pd.DataFrame(
        title_bout_onlyDF, columns=["B_fighter", "B_wins", "B_losses", "B_draw"]
    )

    """Table 2b and Table 3b
    - Name: Title Contender Stats (BodyMeasurements)
    - Columns to include from title_bout_onlyDF: 
      -> R/B_fighter + stats (Height/Reach/Weight/Age/Stance)
    """
    R_TitleStats = pd.DataFrame(
        title_bout_onlyDF,
        columns=[
            "R_fighter",
            "R_Height_cms",
            "R_Reach_cms",
            "R_Weight_lbs",
            "R_age",
            "R_Stance",
        ],
    )
    B_TitleStats = pd.DataFrame(
        title_bout_onlyDF,
        columns=[
            "B_fighter",
            "B_Height_cms",
            "B_Reach_cms",
            "B_Weight_lbs",
            "B_age",
            "B_Stance",
        ],
    )

R_fighter         0
B_fighter         0
Referee          32
Dates             0
location          0
               ... 
R_Height_cms      4
R_Reach_cms     406
R_Weight_lbs      2
B_age           172
R_age            63
Length: 144, dtype: int64


## Creating the databases

- declare_types -> used to automatically convert each column of data from str to its corresponding data type as
                         sqlite3 module has no way to automatically manipulate datetime data (have to convert data from str to its corresponding type 'manually')
                -> .PARSE_COLNAME: scans each column available in tbale
                -> .PARSE_DECLTYPES : scan data type of each column, converts it automatically to its corresponding type

## TitleContender Database

In [132]:
filename = "UFCTitleContendersDB_v1.db"

with closing(
    sqlite3.connect(
        filename, detect_types=sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES
    )
) as ufc_conn:
    with closing(ufc_conn.cursor()) as ufc_cur:
        """creating the tables (using to_sql for now)"""
        """
        TitleBouts.to_sql('TitleContenders', ufc_conn, if_exists = 'fail')
        R_TitleFightRec.to_sql('RedCornerFightRecord', ufc_conn, if_exists = 'fail')
        B_TitleFightRec.to_sql('BlueCornerFightRecord', ufc_conn, if_exists = 'fail')
        R_TitleStats.to_sql('RedCornerStats', ufc_conn, if_exists = 'fail')
        B_TitleStats.to_sql('BlueCornerStats', ufc_conn, if_exists = 'fail')
        """

        # Retrieving all the table names in the database
        tableNames = ufc_cur.execute(
        """
            SELECT name FROM  sqlite_schema
            WHERE type = 'table'
            ORDER BY name
        """
        )
        for n in tableNames:
            print(n)

        # filtering dates via SQL; using the TitleContenders table
        yearInput = str(2020)  # any year between 2000 and 2021
        dateSQL = ufc_cur.execute(
        """
            SELECT * FROM TitleContenders
            WHERE STRFTIME('%Y', Dates) == (?)
        """, (yearInput,),
        )
        # for i in dateSQL:
        # print(i)

        # filtering winners and losers (part 1) -> joining 2 tables
        R_winSQL = ufc_cur.execute(
        """
            SELECT TitleContenders.R_fighter, RedCornerFightRecord.* 
            FROM TitleContenders
                INNER JOIN RedCornerFightRecord ON RedCornerFightRecord.fighterID = TitleContenders.fighterID
                WHERE TitleContenders.Winner = 'Red'
            
        """
        )  # filters Red corner winners from TitleContenders; joins R_fighter column from TitleContenders to RedCornerFightRecord

        B_winSQL = ufc_cur.execute(
        """
            SELECT TitleContenders.B_fighter, BlueCornerFightRecord.* 
            FROM TitleContenders
                INNER JOIN BlueCornerFightRecord ON BlueCornerFightRecord.fighterID = TitleContenders.fighterID
                WHERE TitleContenders.Winner = 'Blue'
        
        """
        )


        # filtering winners and losers (part 2) -> joining 2 tables (FightRec and Stats) while using a third table (TitleContenders) as a conditional

        R_winRecStats = ufc_cur.execute(
        """
            SELECT RedCornerFightRecord.*, RedCornerStats.*
            FROM RedCornerFightRecord
                INNER JOIN TitleContenders
                    ON TitleContenders.fighterID = RedCornerFightRecord.fighterID
                INNER JOIN RedCornerStats
                    ON TitleContenders.fighterID = RedCornerStats.fighterID
            WHERE TitleContenders.Winner = 'Red'
        """
        )


        B_winRecStats = ufc_cur.execute(
        """
            SELECT BlueCornerFightRecord.*, BlueCornerStats.*
            FROM BlueCornerFightRecord
                INNER JOIN TitleContenders 
                    ON BlueCornerFightRecord.fighterID = TitleContenders.fighterID
                INNER JOIN BlueCornerStats 
                    ON TitleContenders.fighterID = BlueCornerStats.fighterID
                WHERE TitleContenders.Winner = 'Blue'
        """
        )
        
        
    ufc_conn.commit()

('BlueCornerFightRecord',)
('BlueCornerStats',)
('RedCornerFightRecord',)
('RedCornerStats',)
('TitleContenders',)
